In [13]:
import geopandas as gpd
import geopy
import geopy.distance
from geopy.geocoders import Nominatim
import folium
import pandas as pd
import numpy as np

Библиотека не обсуждавшаяся в курсе, просто адаптировал.

Получаю координаты по названию места:

In [2]:

geolocator = Nominatim(user_agent="user")
location = geolocator.geocode("Москва", language="ru")
location.point

Point(55.7504461, 37.6174943, 0.0)

In [5]:
url = "https://raw.githubusercontent.com/pinkunicorn322/Final-project-DS/master/postdf.csv"
postdf = pd.read_csv(url).drop(columns=["Unnamed: 0"])
postdf

,city_from,city_to,weight
0,Ростов-на-Дону,Бор,0.300
1,Москва,Волгоград,5.000
2,Сочи,Москва,0.060
3,Добрянка,Пермь,0.100
4,Зерноград,Санкт-Петербург,0.500
...,...,...,...
2616,Москва,Москва,0.075
2617,Москва,Калининград,0.043
2618,Москва,Улан-Удэ,0.029
2619,Москва,Санкт-Петербург,0.033


Для начала преобразуем данные

Следующий блок кода агреггирует все посылки в словарь, который представляет из себя ненаправленный граф. Выводится пример с поселением Бор.

In [24]:
D = {}
for i in postdf.index:
    fr, to, w = postdf.iloc[i, :]
    wedge = tuple(sorted((fr, to)))
    if wedge[0] not in D:
        D[wedge[0]] = {}
        D[wedge[0]][wedge[1]] = w
    else:
        if wedge[1] not in D[wedge[0]]:
            D[wedge[0]][wedge[1]] = w
        else:
            D[wedge[0]][wedge[1]] += w
D["Бор"]

{'Ростов-на-Дону': 0.3}

Чтобы оценить объём всех доставленных посылок вместе с расстоянием, которое они проехали, а также, чтобы показать удобство математическое удобство geopy, посчитаем следующую метрику: Сумма по всем посылкам (расстояние между городами) * (масса посылки)

In [25]:
sm = 0
k = 0
for i in D.keys():
    loc1 = geolocator.geocode(i)
    if loc1 is None:
        continue
    for j in D[i].keys():
        loc2 = geolocator.geocode(j)
        if loc2 is None:
            continue
        sm += geopy.distance.geodesic(loc1.point, loc2.point).km*D[i][j]
        k += 1
        if k % 100 == 0:
            print(k)
print("Значение нашей метрики в 1000 км составило", sm/1000)

100
200
300
400
500
600
700
800
Значение нашей метрики в 1000 км составило 2500.6886646851985


Следующий блок кода выполняется долго(!). Я не стал его перезапускать. Его задача в том, чтобы удалить рёбра с весом <= 2, а также удалить вершины, координаты которых не может найти geopy. Таких координат не много, в основном это небольшие поселения. Как видно, выводил эти места. sys.getsizeof(D) показывает размер словаря в байтах. Я хотел уменьшить D, чтобы карта быстрее рисовалась

In [70]:
for i in list(D.keys()):
    location = geolocator.geocode(i, language="ru")
    
    if location is None:
        del D[i]
        print(i)
        continue
    
    for j in list(D[i].keys()):
        location = geolocator.geocode(j, language="ru")
        if location is None:
            del D[i][j]
            print(j)
            continue
        if D[i][j] <= 2:
            del D[i][j]
sys.getsizeof(D)       

Сиверский, Гатчинский р-н
Новоселье, Ломоносовский р-он
Смольки, Городецкий р-н
Семилуки, Семилукский р-н
Ртищево, Ртищевский р-он
Новосиньково, Дмитровский р-н
Румянцево, Московский пос.
Солнцево, р-н
Шебекино, Шебекинский р-н
Майский, Белгородский р-н
Цемдолина, гор. окр Новороссийск
Семилуки, Семилукский р-н
Ямное, Рамонский р-н
Лиски, Лискинский р-н
Острогожск, Острогожский р-н
Суханово, Ленинский гор.округ
Солнцево, р-н
Тимашевск, Тимашевский р-н
Киров, Кировский р-н
Московский, Тюменский р-н
Миллерово, Миллеровский р-н
Октябрьский, Башкортостан респ.
Коченёво, Коченёвский р-н
Алексеевка, Алексеевский р-н
Васильево, Зеленодольский р-он
Весёлое, Сочи гор.округ
Юбилейный мкр. (Королёв)
Менделеево, Солнечногорский р-он
Румянцево, Московский пос.
Ильский, Северский р-н
Горбунки, Ломоносовский р-н
Гонконг (Чхеклапкок)


9312

И наконец карта. Чем толще ребро, тем больше по массе был объём сдэк перевозок между городами. Как мне кажется, это красиво, и может быть полезно, например, при анализе сферы доставки.

In [75]:
m = folium.Map([55.7504461, 37.6174943], zoom_start=5)
for i in D.keys():
    location = geolocator.geocode(i, language="ru")
#     lat1, lon1 = location.latitude, location.longitude
    if location is None:
        print(i)
        continue
    point1 = tuple([location.latitude, location.longitude])
    
    for j in D[i].keys():
        location = geolocator.geocode(j, language="ru")
        if location is None:
            print(j)
            continue
#         lat2, lon2 = location.latitude, location.longitude
        point2 = tuple([location.latitude, location.longitude])
        folium.PolyLine([point1, point2], color="blue", weight=D[i][j]/5, opacity=0.5).add_to(m)
#         folium.Circle([location.latitude, location.longitude], radius=50).add_to(m)
m

## Итоги: помощь в проверке

1. Pandas - Использовался везде. Из наиболее сложного функция get_dummies, и следующий кусок кода 
        dns_data.groupby(["company"])\
            .count()\
            .sort_values(by="smartphones", ascending=False)\
            .index[:5]
2. Веб-Скреппинг - Использовался Selenium для получения данных с сайта DNS
3. REST API - Использовалось незадокоментированное API для получения данных СДЭК
4. Визуализация - Использовался при работе с данными DNS
5. Математические возможности - спорно, мб подойдёт как я считаю метрику
6. Streamlit - Да
7. Регулярные выражения - Да, при обработке данных DNS
8. Работа с гоеданными - Да, при постройке карты
9. Машинное обучения - Да, предсказывал цены по признакам
10. Работа с графами - Был небольшой анализ графа посылок c помощью networkx
11. Дополнительные технологии - использовал geopy(кажется его не было)
12. Объём - довольно много)()(
13. Целостность - Проект не один, но в меру целостные
14. Общее впечатление - as you wish